## Test GN model equations using Python `PhysicalQuantities` package

The Python `PhysicalQuanties` package enables calculations that take
physical dimensions into account.

This notebook attempts to calculate the GN model's NLI compensation
value while keeping track of units.

For this example, we use a single wavelength,
so we are just looking at self channel interference.

Ultimately we want to make sure that the units are correct,
and compare the results against mininet-optical.

### Imports: math functions and constants; physical quantities

In [76]:
from math import log10, exp, e, pi, asinh
from PhysicalQuantities import q
# Use q.{unit} to disambiguate if necessary
locals().update({f:getattr(q,f) for f in dir(q)})

### Missing units and physical constants

In [77]:
Hz = 1/s
GHz = 1e9 * Hz

# Scaling hacks since units aren't canceling for some reason
nm = 1e-9 * m
km = 1e3 * m
ps = 1e-12 * s

# Physical Constants
Planck = h = 6.62607015e-34 * J/Hz
C = 299792458.0 * m/s

# SMF parameters
fiber_attenuation_db = .22/km
nonlinear_coefficient = 1.27/(W*km)
dispersion = 16.7 * ps/(km*nm)
fiber_attenuation_db, nonlinear_coefficient, dispersion

(0.00022 1/m, 0.00127 1/W/m, 1.6699999999999996e-05 s/m^2)

### Test parameters

In [78]:
pwr = (.001 * dBm).W  # convert dBm to Watts
length = 25 * km
pwr, length

(0.0010002302850208246 W, 25000.0 m)

### Self channel interference, so same power for channel under test and interfering channel

In [79]:
bw = 32 * GHz  # Is this correct?
pwr_cut = pwr
pwr_ch = pwr_cut
bw_cut = bw
bw_ch = bw
pwr_cut, bw_cut

(0.0010002302850208246 W, 32000000000.0 1/s)

In [89]:
alpha = fiber_attenuation_db / (20*log10(e))
asymptotic_length = 1 / (2*alpha)
effective_length = ((1 - exp(-2 * alpha * length)) / 
                    (2 * alpha))
ref_wavelength = 1550 * nm

D = dispersion
beta2 = ref_wavelength**2 * D / (2 * pi * C)

effective_length, D, gamma

(14176.984836791155 m, 1.6699999999999996e-05 s/m^2, 0.00127 1/W/m)

### Constant term from GN-model equation

In [91]:
gamma = nonlinear_coefficient
cterm = ((16/27) * (gamma * effective_length)**2 /
         (2*pi * beta2 * asymptotic_length))
cterm

7.271275754823355e+22 $\frac{1}{\text{W}^2\cdot \text{s}^2}$

###  Self channel interference psi computation
This value should be unitless

In [82]:
psi = asinh(0.5 * pi*pi * asymptotic_length * beta2 * bw_cut*bw_cut)
psi

1.4980786903221437

### g is the flat PSD for the given channel
nli is the GN-model's NLI power compensation term, and should be in Watts

In [87]:
g_cut = pwr_cut/bw_cut
g_ch = pwr_ch/bw_ch
g_nli = cterm * (g_ch*g_ch) * g_cut * psi
nli = g_nli*bw_cut
g_cut, g_ch, g_nli, nli

(3.125719640690077e-14 W*s,
 3.125719640690077e-14 W*s,
 3.3265595818736133e-18 W*s,
 1.0644990661995562e-07 W)